# Анализ ошибок бейзлайна

In [1]:
import pandas as pd
import sys
import os

# Добавляем путь к проекту, чтобы можно было импортировать из utils
try:
    CURRENT_DIR = os.path.dirname(os.path.abspath(__file__))
except NameError:
    CURRENT_DIR = os.getcwd()
    
BASE_DIR = os.path.abspath(os.path.join(CURRENT_DIR, ".."))
if BASE_DIR not in sys.path:
    sys.path.insert(0, BASE_DIR)
from utils.config import DATA_PATH,EXPERIMENTS_DIR
from utils.inspector import inspect_row
from utils.data_loader import load_dataset
from utils.unify_columns import unify_df

In [2]:
df = pd.read_csv(os.path.join(EXPERIMENTS_DIR, "baseline_val_predictions_old.csv"))
df = unify_df(df)
df.shape

(598, 9)

In [3]:
df.head(5)

,text,address,name,normalized_main_rubric_name_ru,permalink,prices_summarized,relevance,reviews_summarized,pred_relevance
0,анализ крови по какому формеру на сифилис,"Санкт-Петербург, Ленинский проспект, 121",KDL,Медицинская лаборатория,200967546138,Медицинская лаборатория KDL предоставляет широ...,1.0,Организация занимается проведением анализов и ...,1.0
1,стретчинг в москве,"Москва, Кастанаевская улица, 39",Elasti_city; Elasti city,Фитнес-клуб,147723830462,"Фитнес-клуб и студия йоги, предлагающие пробны...",0.0,Организация занимается предоставлением услуг ф...,1.0
2,стейк ресторан на белорусской,"Москва, 2-й Тверской-Ямской переулок, 10",Доктор на крыше; Doktor na kryshe; Доктор; Dok...,Ресторан,1081999210,Ресторан «Доктор на крыше» предлагает разнообр...,0.0,Организация «Доктор на крыше» предлагает услуг...,0.0
3,дежурная стоматология севастополь,"Севастополь, Античный проспект, 8В",Атмосфера; Atmosfera; Стоматология Атмосфера; ...,Стоматологическая клиника,82384383021,Стоматологическая клиника «Атмосфера» предоста...,0.0,Организация занимается стоматологическими услу...,1.0
4,суши,"Московская область, городской округ Пушкинский...",Subway,Быстрое питание,240244902340,Subway предлагает широкий ассортимент сэндвиче...,0.0,"Организация занимается быстрым питанием, предо...",0.0


In [4]:
train_data, val_data, test_data = load_dataset(DATA_PATH, drop_uncertain=True, val_frac=0.01)

In [5]:
df.relevance.value_counts()

relevance
1.0    335
0.0    263
Name: count, dtype: int64

In [6]:
df.pred_relevance.value_counts()

pred_relevance
1.0    385
0.0    213
Name: count, dtype: int64

In [7]:
acc = (df["relevance"] == df["pred_relevance"]).mean()
print("Accuracy:", round(acc, 4))

# 4. Анализ ошибок
false_positives = df[(df["relevance"] == 0.0) & (df["pred_relevance"] == 1.0)]
false_negatives = df[(df["relevance"] == 1.0) & (df["pred_relevance"] == 0.0)]

# 5. Сохранение для ручного анализа
false_positives.to_csv("false_positives.csv", index=False)
false_negatives.to_csv("false_negatives.csv", index=False)

print(f"False Positives: {len(false_positives)}")
print(f"False Negatives: {len(false_negatives)}")

Accuracy: 0.6488
False Positives: 130
False Negatives: 80


Модель склонна переоценивать релевантность — она часто предсказывает RELEVANT_PLUS, даже когда это не так. (130 FP — много ложных позитивов.)

In [6]:
# Создаем метки ошибок
df['error_type'] = 'correct'
df.loc[(df['relevance'] == 0.0) & (df['gpt_response'] == 'RELEVANT_PLUS'), 'error_type'] = 'false_positive'
df.loc[(df['relevance'] == 1.0) & (df['gpt_response'] == 'IRRELEVANT'), 'error_type'] = 'false_negative'

df[df['error_type'] == 'false_positive']
df[df['error_type'] == 'false_negative']

false_pos_df = df[df['error_type'] == 'false_positive']
false_neg_df = df[df['error_type'] == 'false_negative']

# Сохраняем отдельные ошибки
false_pos_df.to_csv("baseline_false_positives.csv", index=False)
false_neg_df.to_csv("baseline_false_negatives.csv", index=False)

In [8]:
false_pos_df['normalized_main_rubric_name_ru'].value_counts().head(10)

Кафе                                   9
Гостиница                              6
Медцентр , клиника                     6
Ресторан                               6
Автосалон                              5
Салон красоты                          3
Быстрое питание                        3
Столовая                               3
Магазин автозапчастей и автотоваров    3
Детский сад , ясли                     3
Name: normalized_main_rubric_name_ru, dtype: int64

In [9]:
false_pos_df = false_pos_df.copy()     
false_pos_df['text_length'] = false_pos_df['text'].str.len()
false_pos_df['reviews_length'] = false_pos_df['reviews_summarized'].str.len()
false_pos_df[['text_length', 'reviews_length']].describe()

,text_length,reviews_length
count,130.000000,125.00000
mean,27.076923,1615.92800
std,15.217093,555.41538
min,3.000000,167.00000
25%,16.000000,1290.00000
50%,25.000000,1715.00000
75%,34.000000,1981.00000
max,100.000000,2864.00000


In [174]:
false_pos_df[false_pos_df.text_length<=27].head(100)[50:80]

,text,address,name,normalized_main_rubric_name_ru,permalink,prices_summarized,relevance,reviews_summarized,gpt_response,gpt_pred_relevance,error_type,text_length,reviews_length
477,фгос дошкольники,"Москва, Трифоновская улица, 51А, стр. 2",Детский сад РЖД № 5; РЖД детский сад № 5; Детс...,"Детский сад , ясли",29451943660,NaN,0.0,Детский сад РЖД № 5 хвалят за профессионализм ...,RELEVANT_PLUS,1.0,false_positive,16,2524.0
478,клубные рестораны в спб,"Санкт-Петербург, проспект Энгельса, 124, корп. 1",Хаус; Haus bar; Hausbar; Караоке бар Хаус; Рес...,Караоке-клуб,1190255856,"Хаус предлагает широкий выбор блюд, коктейлей ...",0.0,"Организация «Хаус» — это караоке-клуб, ночной ...",RELEVANT_PLUS,1.0,false_positive,23,1092.0
480,печать наклеек,"Ростовская область, Батайск, улица Кирова, 9Б",Панорама; Panorama,Фотоуслуги,242567051243,Организация «Панорама» предоставляет фотоуслуг...,0.0,"Организация предоставляет фотоуслуги, широкофо...",RELEVANT_PLUS,1.0,false_positive,14,1727.0
481,пицца в сургуте,"Ханты-Мансийский автономный округ — Югра, Сург...",Точка вкуса; Tochka-vkusa; Точка Вкуса,Доставка еды и обедов,20536000056,«Точка вкуса» предлагает широкий ассортимент с...,0.0,"Организация занимается доставкой еды и обедов,...",RELEVANT_PLUS,1.0,false_positive,15,1733.0
484,френдс суши самара,"Самара, Молодогвардейская улица, 135",Фуджи; Fuji; Friend Фуджи Суши; Friends Фуджи ...,Суши-бар,1327093655,Суши-бар и пиццерия «Фуджи» предлагает широкий...,0.0,Организация занимается приготовлением и достав...,RELEVANT_PLUS,1.0,false_positive,18,1769.0
488,наращивание ресниц,"Москва, проспект 60-летия Октября, 16к1",Мэйд Ю Нэйлс; Made Your Nails; MadeY; Мэйд Ю,Ногтевая студия,20834960525,Салон красоты «Мэйд Ю Нэйлс» предоставляет усл...,0.0,Организация занимается ногтевым сервисом и про...,RELEVANT_PLUS,1.0,false_positive,18,1749.0
491,фарментин 400,"Смоленская область, Сафоново, улица Ленина, 39",Максавит; Maksavit; Аптека Максавит,Аптека,14020585155,Аптека «Максавит» предлагает широкий ассортиме...,0.0,Организация занимается продажей лекарственных ...,RELEVANT_PLUS,1.0,false_positive,13,1477.0
493,суши бар рязань,"Рязань, Татарская улица, 95",Автосуши Автопицца; Avtosushi Avtopizza; Автоп...,Суши-бар,171672052301,Автосуши Автопицца предлагает широкий выбор пи...,0.0,Организация занимается приготовлением и достав...,RELEVANT_PLUS,1.0,false_positive,15,1211.0
504,Аутлет внуково,"Москва, Новомосковский административный округ,...",Högl Outlet,Магазин обуви,61183237818,NaN,0.0,NaN,RELEVANT_PLUS,1.0,false_positive,14,NaN
519,ожоговая больница москва,"Москва, Фортунатовская улица, 1, корп. 10","Департамент Здравоохранения, ГКБ им. Ф. И. Ино...",Больница для взрослых,44259489914,NaN,0.0,Организация занимается оказанием медицинской п...,RELEVANT_PLUS,1.0,false_positive,24,1559.0


In [11]:
false_pos_df[false_pos_df.text_length==100]

,text,address,name,normalized_main_rubric_name_ru,permalink,prices_summarized,relevance,reviews_summarized,gpt_response,gpt_pred_relevance,error_type,text_length,reviews_length
191,FIA07DU010 EUROBUMP Бампер задн лев с катафото...,"Москва, Кетчерская улица, 2А",AvtoAll; АвтоАльянс; Автозапчасти в Новогиреев...,Магазин автозапчастей и автотоваров,1197606102,AvtoAll предлагает широкий ассортимент автозап...,0.0,Организация занимается продажей автозапчастей ...,RELEVANT_PLUS,1.0,false_positive,100,922.0


In [190]:
train_data[train_data.text.str.contains('наращивание')]

,text,address,name,normalized_main_rubric_name_ru,permalink,prices_summarized,relevance,reviews_summarized
12,наращивание ногтей екатеринбург брилиант,"Свердловская область, Екатеринбург, проспект Л...",Лаки; Laku; Lаки,Ногтевая студия,229494192641,Ногтевая студия «Лаки» предлагает широкий спек...,0.0,Организация занимается предоставлением услуг н...
140,школа наращивание ногтей свао,"Москва, Берёзовая аллея, 17, корп. 2",Top Nails,Салон красоты,152528927368,Top Nails предлагает широкий спектр услуг по у...,0.0,Организация занимается предоставлением услуг в...
2047,наращивание ресниц новокузнецк,"Кемеровская область, Новокузнецк, Запорожская ...",Agapova Studio,Салон красоты,171314439639,Agapova Studio предлагает широкий спектр услуг...,0.0,Организация занимается предоставлением услуг в...
3619,наращивание ресниц на Выхино,"Москва, Святоозёрская улица, 26",RMnails,Ногтевая студия,172336696376,"RMnails предлагает услуги по уходу за ногтями,...",0.0,"Организация занимается ногтевым сервисом, уход..."
3735,наращивание ресниц на Выхино,"Москва, Ферганская улица, 14/13",Aishe Nails,Салон красоты,171357307847,Салон красоты Aishe Nails предлагает услуги по...,1.0,Организация занимается предоставлением услуг м...
3962,наращивание ресниц на Выхино,"Москва, Ферганский проезд, 10Б, стр. 2",Леди-Ди; Lady Dee; Салон красоты Леди-Di; Ledi...,Салон красоты,1237530275,Салон красоты «Леди-Ди» предоставляет широкий ...,1.0,Организация занимается предоставлением услуг в...
5608,наращивание ресниц,"Саратов, улица имени Т.Г. Шевченко, 54",Блэк даймонд; Black diamond; Black Diamond,Салон красоты,188721894034,Салон красоты «Блэк даймонд» предлагает широки...,1.0,Организация занимается предоставлением услуг в...
5758,наращивание ресниц на Выхино,"Москва, Вешняковская улица, 18Б",Beauty Lab; Салон красоты,Салон красоты,108417752628,Beauty Lab предлагает услуги стрижки для детей...,1.0,Организация занимается предоставлением услуг в...
7610,обучение наращивание ресниц челябинск,"Челябинск, Комсомольский проспект, 30","Wildberries; Wildberries.ru; Wildberries.ru, п...",Пункт выдачи,80401016904,None,0.0,Организация занимается выдачей заказов интерне...
8262,наращивание ресниц,"Саратов, улица имени И.В. Мичурина, 55/61",Tao Balance,Массажный салон,97839532108,Tao Balance предлагает массажные и спа-процеду...,0.0,Организация занимается предоставлением массажн...


In [8]:
inspect_row(df, 570, pred_col="pred_relevance", label_col='relevance')

In [199]:
inspect_row(train_data, 10855)

# False Positives

In [142]:
from IPython.display import display, HTML

def inspect_row(df, idx):
    """HTML-вывод ключевых полей строки для анализа ошибок классификации."""
    row = df.loc[idx]

    html = f"""
    <div style="border:1px solid #ccc; padding:16px; border-radius:10px; font-family:sans-serif; background-color:#f9f9f9">
        <h2>📌 Index: {idx}</h2>
        <p><strong>Запрос:</strong><br>{row['text']}</p>
        <p><strong>Адрес:</strong><br>{row['address']}</p>
        <p><strong>Название:</strong><br>{row['name']}</p>
        <p><strong>Рубрика:</strong><br>{row['normalized_main_rubric_name_ru']}</p>
        <p><strong>Описание:</strong><br>{row['prices_summarized']}</p>
        <p><strong>Relevance:</strong> {row['relevance']}</p>
        <p><strong>Отзывы:</strong><br>{row['reviews_summarized']}</p>
        <p><strong>GPT ответ:</strong> {row.get('gpt_response', 'N/A')}<br>
           <strong>GPT предсказание:</strong> {row.get('gpt_pred_relevance', 'N/A')}</p>
    </div>
    """

    display(HTML(html))

## Ошибки асессоров

### ?

In [144]:
inspect_row(false_pos_df, 54)

In [145]:
inspect_row(false_pos_df,111)

In [146]:
inspect_row(false_pos_df,126)

In [147]:
inspect_row(false_pos_df,163)

In [148]:
inspect_row(false_pos_df,182)

In [149]:
inspect_row(false_pos_df,211)

In [150]:
inspect_row(false_pos_df,271)

In [151]:
inspect_row(false_pos_df,316)

In [152]:
inspect_row(false_pos_df,301)

In [153]:
train_data[((train_data['text'] == 'пиво разливное') | (train_data['text'] == 'разливное пиво')) & 
     (train_data['reviews_summarized'].str.contains('разливн.+пив', na=False)) & 
     (train_data['normalized_main_rubric_name_ru'] == 'Магазин пива')]['relevance'].value_counts()

1.0    6
0.0    2
Name: relevance, dtype: int64

Скорее, ошибка асессора судя по разбросу relevance в обучающих данных. 

In [170]:
inspect_row(false_pos_df,459)

In [178]:
inspect_row(false_pos_df, 481)

In [177]:
inspect_row(false_pos_df,484)

## Ошибки модели

#### Необходимо искать информацию об организации, в частности о часах работы
Вот тут нужно тулз, чтобы сделать запрос в гугл "адрес название организации часы работы"

In [168]:
inspect_row(false_pos_df,454)

In [154]:
inspect_row(false_pos_df,37)

In [155]:
inspect_row(false_pos_df,372)

In [171]:
inspect_row(false_pos_df,468)

In [156]:
inspect_row(false_pos_df,171)

#### Возможно, если указана какая-то конкретная организация или бренд, то нужно искать ее упоминание в предоставленной информации

In [159]:
inspect_row(false_pos_df,186)

#### Обращать внимание на нумерацию организаций

In [167]:
inspect_row(false_pos_df,414)

#### Уточнить промт - просить искать в prices_summarized, если поле не пустое. Искать конкретные услуги\товары в описании, если они есть в запросе?

In [160]:
inspect_row(false_pos_df,218)

* если запрос - какая-то медицинская процедура, то она в явном виде должна быть указана в описании организации

In [161]:
inspect_row(false_pos_df,242)

In [166]:
inspect_row(false_pos_df,403)

In [157]:
inspect_row(false_pos_df,317)

In [176]:
inspect_row(false_pos_df, 480)

### не игнорировать слово "частный"

In [172]:
inspect_row(false_pos_df, 467)

#### Если в запросе есть части адреса, тщательно проверить адрес организации

In [162]:
inspect_row(false_pos_df,355)

#### Малоинформативный запрос. 

В случае малоинформативного запроса типа "ремонт" - предсказывать по умолчанию IRRELEVANT...

In [163]:
inspect_row(false_pos_df,124)

In [164]:
inspect_row(false_pos_df,434)

In [175]:
inspect_row(false_pos_df,477)

### не ясно пока

In [143]:
inspect_row(false_pos_df, 15)

### False Negatives

## Ошибки модели

In [220]:
false_neg_df.head(20)

,text,address,name,normalized_main_rubric_name_ru,permalink,prices_summarized,relevance,reviews_summarized,gpt_response,gpt_pred_relevance,error_type
14,где вкусно и недорого поесть на невском проспе...,"Санкт-Петербург, проспект Просвещения, 19",Вкусно — и точка; Vkusno — i tochka; McDonald'...,Быстрое питание,209986321255,«Вкусно — и точка» предлагает широкий ассортим...,1.0,"Организация занимается быстрым питанием, анало...",IRRELEVANT,0.0,false_negative
25,где дешево поесть в санкт-петербурге 2017,"Санкт-Петербург, Малая Морская улица, 7",Буше; Bushe; БУШЕ пекарня-кондитерская ООО; Бу...,Пекарня,1023261511,Пекарня «Буше» предлагает широкий ассортимент ...,1.0,Организация занимается выпечкой и продажей кон...,IRRELEVANT,0.0,false_negative
39,тетеплый плинтус 30мм высота на профиль окна,"Воронежская область, Рамонский район, Яменское...",Лемана Про; Lemana Pro; Leroy Merlin; Леруа Ме...,Строительный магазин,1583592856,Лемана Про предлагает широкий ассортимент това...,1.0,Организация «Лемана Про» занимается продажей с...,IRRELEVANT,0.0,false_negative
43,зал для кроссфита,"Республика Башкортостан, Уфа, бульвар Тухвата ...",Лейтай; Leitai,"Спортивный клуб , секция",60891450032,Спортивный клуб «Лейтай» предлагает занятия в ...,1.0,Организация «Лейтай» занимается проведением сп...,IRRELEVANT,0.0,false_negative
64,рестораны в тц океания,"Москва, Мироновская улица, 33, стр. 28",Кавказский дворик; Kavkazsky Dvorik,Ресторан,1004953355,Ресторан «Кавказский дворик» предлагает широки...,1.0,Организация занимается предоставлением услуг в...,IRRELEVANT,0.0,false_negative
65,национальный парк стран мира,"Калужская область, Боровский район, сельское п...",Этнографический парк-музей Этномир; Ethnograph...,Парк культуры и отдыха,44152522005,NaN,1.0,Организация занимается этнографическим туризмо...,IRRELEVANT,0.0,false_negative
69,автосалон лада,"Москва, Сормовская улица, 21А",АвтоГЕРМЕС сервис УАЗ; AvtoGERMES Uaz; АвтоГЕР...,"Автосервис , автотехцентр",120603898139,АвтоГЕРМЕС сервис УАЗ предоставляет услуги по ...,1.0,АвтоГЕРМЕС сервис УАЗ занимается обслуживанием...,IRRELEVANT,0.0,false_negative
71,текстиль метро,"Москва, шоссе Энтузиастов, 12, корп. 2",Домовой; Domovoj; Домовой Город; Domovoy,Товары для дома,212544904787,Магазин «Домовой» предлагает широкий ассортиме...,1.0,Организация «Домовой» занимается продажей това...,IRRELEVANT,0.0,false_negative
79,рестораны москва сити,"Москва, улица Большая Дмитровка, 7/5с2",Tehnikum; Техникум; Бистро Tehnikum; Ресторан ...,Ресторан,1753926793,Ресторан Tehnikum предлагает широкий выбор блю...,1.0,Организация занимается ресторанным бизнесом в ...,IRRELEVANT,0.0,false_negative
96,франшиза итальянского ресторана,"Москва, улица Большая Полянка, 30",Сиеста вино & паста; Siesta wine & pasta; Кули...,Ресторан,95131072317,Ресторан «Сиеста вино & паста» предлагает широ...,1.0,Организация «Сиеста вино & паста» занимается р...,IRRELEVANT,0.0,false_negative


## Ошибки асессоров

In [244]:
# запрос htvjyn dtkjcbgtljd
inspect_row(false_neg_df, 136)

Вопрос не к Яндекс.Картам:))

In [246]:
inspect_row(false_neg_df, 170)

Пивнушка Дурдин была по другому адресу и это вообще не метро Сокол...

In [245]:
inspect_row(false_neg_df, 149)

In [243]:
inspect_row(false_neg_df, 104)

In [242]:
inspect_row(false_neg_df, 64)

In [241]:
inspect_row(false_neg_df, 43)

In [219]:
inspect_row(false_neg_df, 14)

В отзыве четко описано "Считают цены высокими"!

In [221]:
inspect_row(false_neg_df, 25)

In [222]:
inspect_row(false_neg_df, 39)

In [249]:
train_data, val_data, test_data = load_dataset(DATA_PATH, drop_uncertain=True, val_frac=0.01)

In [252]:
test_data.isnull().sum()

text                                0
address                             0
name                                0
normalized_main_rubric_name_ru      0
permalink                           0
prices_summarized                 207
relevance                           0
reviews_summarized                 15
dtype: int64

In [29]:
train_data[train_data.text.str.contains(r'метро')]

,text,address,name,normalized_main_rubric_name_ru,permalink,prices_summarized,relevance,reviews_summarized,relevance_new
9,новостройки москвы с отделкой от застройщика р...,"Москва, Ленинградский проспект, 80к16, подъезд 3",Миэль; Miel; МИЭЛЬ; Офис на Соколе,Агентство недвижимости,1227767207,None,0.0,Организация занимается сделками с недвижимость...,0.0
53,расположение станций метро в новой москве на к...,"Москва, Московское центральное кольцо, станция...",Крымская; Krymskaya; Кримська; Метро Крымская;...,Станция метро,121441159673,None,1.0,Организация представляет собой станцию метро М...,1.0
103,Ремонт глюкометров томск,"Томск, проспект Фрунзе, 232",Дельрус,"Медицинское оборудование , медтехника",76170150859,None,0.0,None,0.0
238,мрт в метро,"Москва, Ленинградское шоссе, 47",Медскан; Medical Center Medskan; Medscan; Диаг...,"Медцентр , клиника",1715217562,Медскан предоставляет широкий спектр физиотера...,1.0,Организация занимается медицинскими обследован...,1.0
886,суточный квартиры метро шелепиха,"Москва, Новомосковский административный округ,...",Суточная квартира; Apartments; Сутачная Кватэр...,Жильё посуточно,242854723031,Предлагает аренду однокомнатной квартиры посут...,1.0,Организация предоставляет жильё посуточно. Отз...,1.0
...,...,...,...,...,...,...,...,...,...
27834,сервисный центр and в москве ремонт тонометров...,"Москва, Вересковая улица, 16",Ремсервисбыт; Remservisbyut; Remservisbyt; Сти...,Ремонт бытовой техники,235281450820,"Ремонт бытовой техники, включая стиральные, су...",0.0,Организация занимается ремонтом бытовой техник...,0.0
28180,Фото на документы метро академическая,"Санкт-Петербург, улица Бабушкина, 12",Фотосет; FotoSet; Фото Сет; Фото копи центр; Ф...,Фотоуслуги,1178912561,"Предоставляет фотоуслуги, услуги печати и копи...",0.0,"Организация предоставляет фотоуслуги, печать и...",0.0
29133,Москва станция метро Парк Победы отели поблизости,"Тульская область, Щёкинский район, сельское по...",Слобода; Sloboda; База отдыха Слобода,Гостевой дом,153146857059,Гостевой дом «Слобода» предлагает различные ва...,1.0,Гостевой дом «Слобода» предлагает проживание в...,1.0
29160,недорогое кафе в москве рядом с метро,"Москва, Яузский бульвар, 14/8",47+Жöн; Lozhechka; Ложечка; +47; Ложка; Lozhka,Ресторан,20652000034,None,0.0,Организация занимается предоставлением услуг р...,0.0


In [31]:
train_data.loc[886]

text                                               суточный квартиры метро шелепиха
address                           Москва, Новомосковский административный округ,...
name                              Суточная квартира; Apartments; Сутачная Кватэр...
normalized_main_rubric_name_ru                                      Жильё посуточно
permalink                                                              242854723031
prices_summarized                 Предлагает аренду однокомнатной квартиры посут...
relevance                                                                       1.0
reviews_summarized                Организация предоставляет жильё посуточно. Отз...
relevance_new                                                                   1.0
Name: 886, dtype: object

In [32]:
train_data.loc[886]['address']

'Москва, Новомосковский административный округ, посёлок Коммунарка, улица Потаповская Роща, 7, корп. 1'